In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
import urllib, json

# 행정동 데이터 전처리

In [61]:
# 유동인구 데이터 불러오기

f = pd.read_csv("fpopl.csv")
df = pd.DataFrame(f)
df.shape

(47546443, 6)

In [62]:
# 행정동 데이터 불러오기

f2 = pd.read_csv("adstrd_master.csv")
df2 = pd.DataFrame(f2)

df2.shape

(3493, 4)

In [63]:
# df와 df2 합치기 (유동인구['adstrd_code'] = 행정동['adstrd_code'])
#                          * adstrd_code = 행정동코드
# 유동인구 데이터와 행정동 이름 합침
# SQL의 LEFT JOIN

move_people = pd.merge(df,df2,on="adstrd_code",how="left")
move_people.head()

,base_ymd,tmzon_se_code,sexdstn_se_code,agrde_se_code,adstrd_code,popltn_cascnt,adstrd_nm,brtc_nm,signgu_nm
0,20200101,0,M,age_00,11650560,15,반포1동,서울특별시,서초구
1,20200101,0,M,age_00,11590620,6,사당1동,서울특별시,동작구
2,20200101,0,M,age_00,11560710,4,대림2동,서울특별시,영등포구
3,20200101,0,M,age_00,11470680,12,신정7동,서울특별시,양천구
4,20200101,0,M,age_00,11350665,6,상계3.4동,서울특별시,노원구


In [64]:
del move_people['brtc_nm']

# brtc_nm은 시도 명이다.
# 현 DATASET은 서울시 내 행정구역 데이터이기 때문에, brtc_nm 칼럼 삭제

In [65]:
move_people = move_people.rename({'base_ymd':'년월일','tmzon_se_code':'시간대','sexdstn_se_code':'성별','agrde_se_code':'나이대','adstrd_code':'행정동코드','popltn_cascnt':'유동인구','adstrd_nm':'행정동','signgu_nm':'행정구'}, axis ='columns')

grouped = move_people.groupby(by = ['년월일','행정구']).sum()['유동인구'].reset_index()

grouped

,년월일,행정구,유동인구
0,20200101,강남구,3028808
1,20200101,강동구,2094980
2,20200101,강북구,945218
3,20200101,강서구,2191177
4,20200101,관악구,1694584
...,...,...,...
4195,20200616,용산구,2363994
4196,20200616,은평구,1610731
4197,20200616,종로구,1992243
4198,20200616,중구,2253617


In [66]:
temp = grouped[grouped.년월일==20200408]


# 지도 그리기

## 서울 자치구 경계 json파일 다운로드

In [67]:
url = 'https://raw.github.com/PinkWink/DataScience/master/data/02.%20skorea_municipalities_geo_simple.json'
with urllib.request.urlopen(url) as fp:
  json_seoul = json.loads(fp.read())


In [68]:
for idx, seoul_dict in enumerate(json_seoul['features']):
  gu_name = seoul_dict['id']
  value = temp.loc[temp.행정구==gu_name, '유동인구'].iloc[0]
  txt = f'{gu_name}<br>{value}'
  json_seoul['features'][idx]['properties']['tooltip1'] = txt

In [69]:
m = folium.Map(location=[37.55, 127], zoom_start=11, tiles='Stamen Terrain')


In [70]:
choropleth = folium.Choropleth(
    geo_data=json_seoul, 
    data=temp,
    columns=['행정구', '유동인구'],
    fill_color = 'OrRd',
    key_on = 'feature.id',
    fill_opacity=0.9,
    line_opacity=1,
    legend_name='Foot Traffic'
).add_to(m)

choropleth.geojson.add_child( folium.features.GeoJsonTooltip(['tooltip1'], labels=False))

title_html = '<h3 align="center" style="font-size:20px"><b>Foot Traffic in Seoul</b></h3>'
m.get_root().html.add_child(folium.Element(title_html))


folium.LayerControl().add_to(m)

m

In [71]:
import geopandas as gpd

df = gpd.read_file('HangJeongDong_ver20200401')

ModuleNotFoundError: No module named 'geopandas'

In [72]:
!pip install geopandas

    ERROR: Command errored out with exit status 1:
     command: 'C:\ProgramData\Anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\COM\\AppData\\Local\\Temp\\pip-install-lg01bt6v\\fiona\\setup.py'"'"'; __file__='"'"'C:\\Users\\COM\\AppData\\Local\\Temp\\pip-install-lg01bt6v\\fiona\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\COM\AppData\Local\Temp\pip-pip-egg-info-_w2zuz00'
         cwd: C:\Users\COM\AppData\Local\Temp\pip-install-lg01bt6v\fiona\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
    ----------------------------------------
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.



  Using cached geopandas-0.8.1-py2.py3-none-any.whl (962 kB)
  Using cached Fiona-1.8.13.post1.tar.gz (1.2 MB)
